In [43]:
import scipy.stats as stats
import numpy as np
import tensorflow.keras.backend as K


In [96]:
MAX_SEQ_LEN = 6

In [121]:
def calulateKendallTau(y_true, y_pred):
    y_true = K.eval(y_true)
    y_pred = K.eval(y_pred)
    
    y_true_indexed = (np.argmax(y_true, axis=-1) + 1)
    y_pred_indexed = (np.argmax(y_pred, axis=-1) + 1)
    
    
    corrected_y_pred_indexed = (y_true_indexed != MAX_SEQ_LEN) * y_pred_indexed
    corrected_y_pred_indexed[corrected_y_pred_indexed == 0] = MAX_SEQ_LEN
    
    kendal_tau_list = []
    for i in range(y_true_indexed.shape[0]):
        print(stats.kendalltau(y_true_indexed[i],corrected_y_pred_indexed[i])[0])
        kendal_tau_list.append(stats.kendalltau(y_true_indexed[i],corrected_y_pred_indexed[i])[0])
    
    return np.mean(kendal_tau_list)
    